# Weeks 8-9: Brownian Motion

**Objective:** Understand Brownian motion as the continuous-time limit of a random walk, learn its defining properties, and simulate its paths to visualize its behavior.

## Step 1: Build Intuition

Imagine a tiny particle, like a grain of pollen, suspended in water. When you look at it under a microscope, you'll see it moving around in a random, jittery, and unpredictable way. This is because it's constantly being bombarded by water molecules. This phenomenon, first observed by Robert Brown, is the classic image of **Brownian motion**.

Think of it as a **random walk in continuous time**. Instead of taking a discrete step every second, the particle is taking infinitely many, infinitesimally small random steps at all times. The result is a path that is continuous (it doesn't teleport), but also incredibly erratic and "jagged".

## Step 2: Understand the Core Idea

Brownian motion (also called a **Wiener Process**) is a specific type of continuous-time stochastic process, denoted \(W(t)\) or \(W_t\). Its behavior is defined by a few key properties:

1.  **Starts at Zero:** The process always starts at the origin: \(W(0) = 0\).
2.  **Continuous Paths:** The path \(W(t)\) is continuous everywhere. There are no jumps.
3.  **Independent Increments:** The change in the process over one time interval, \(W(t) - W(s)\), is independent of the change over any other *non-overlapping* interval, \(W(v) - W(u)\).
4.  **Normal, Stationary Increments:** The change over any time interval of length \(s\) is normally distributed with a mean of 0 and a variance equal to the length of the interval, \(s\). 
    - That is, \(W(t+s) - W(t) \sim \mathcal{N}(0, s)\).
    - This also means \(W(t) \sim \mathcal{N}(0, t)\), since \(W(t) = W(t) - W(0)\).

## Step 3: Learn the Definitions and Formulas

**Definition: Standard Brownian Motion (or Wiener Process)**
A stochastic process \({W(t), t \ge 0}\) is a standard Brownian motion if it satisfies:
1.  \(W(0) = 0\).
2.  With probability 1, the function \(t \mapsto W(t)\) is continuous in \(t\).
3.  The process has independent increments.
4.  For any \(s < t\), the increment \(W(t) - W(s)\) is a random variable from a normal distribution with mean 0 and variance \(t-s\): \(W(t) - W(s) \sim \mathcal{N}(0, t-s)\).

**Is Brownian Motion a Martingale?**
Yes! A standard Brownian motion is a martingale. We can check the condition \(E[W(t) | \mathcal{F}_s] = W(s)\) for \(s < t\).
\(E[W(t) | \mathcal{F}_s] = E[W(t) - W(s) + W(s) | \mathcal{F}_s]\)
\(= E[W(t) - W(s) | \mathcal{F}_s] + E[W(s) | \mathcal{F}_s]\)
- Because of independent increments, \(E[W(t) - W(s) | \mathcal{F}_s] = E[W(t) - W(s)] = 0\).
- Because \(W(s)\) is known at time \(s\), \(E[W(s) | \mathcal{F}_s] = W(s)\).
So, \(E[W(t) | \mathcal{F}_s] = 0 + W(s) = W(s)\). It satisfies the martingale property.

## Step 4: Apply and Practice

We can't perfectly simulate a continuous process on a computer. Instead, we approximate it by taking very small discrete time steps. This is called the **Euler-Maruyama method**.

If we divide our time horizon \(T\) into \(N\) small steps of size \(\Delta t = T/N\), then the change in the process over one small step is:
$$ \Delta W_t = W(t + \Delta t) - W(t) \sim \mathcal{N}(0, \Delta t) $$
We can rewrite this as \(\Delta W_t = Z \sqrt{\Delta t}\), where \(Z \sim \mathcal{N}(0, 1)\) is a standard normal random variable.

This gives us a simple update rule for simulation:
$$ W(t_{i+1}) = W(t_i) + Z_i \sqrt{\Delta t} $$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

plt.style.use('seaborn-v0_8-whitegrid')

### Part A: Simulating and Visualizing Brownian Motion Paths

In [ ]:
def simulate_brownian_motion(T, N):
    """
    Simulates a path of standard Brownian motion.
    
    Args:
        T (float): The time horizon.
        N (int): The number of time steps.
        
    Returns:
        tuple: A tuple containing (time_axis, path).
    """
    dt = T / N
    # Generate the random increments, Z * sqrt(dt)
    # Z ~ N(0, 1), so increments ~ N(0, dt)
    increments = np.random.normal(loc=0, scale=np.sqrt(dt), size=N)
    
    # Prepend 0 for the starting point and take the cumulative sum
    path = np.concatenate(([0], increments)).cumsum()
    
    time_axis = np.linspace(0, T, N + 1)
    
    return time_axis, path

# --- Simulation Parameters ---
TIME_HORIZON = 1.0  # Simulate for 1 unit of time
NUM_STEPS = 500     # Number of time steps
NUM_PATHS = 5       # Number of different paths to visualize

# --- Plotting ---
plt.figure(figsize=(14, 8))

for _ in range(NUM_PATHS):
    t, path = simulate_brownian_motion(TIME_HORIZON, NUM_STEPS)
    plt.plot(t, path)

plt.title(f'{NUM_PATHS} Simulated Paths of Brownian Motion')
plt.xlabel('Time (t)')
plt.ylabel('Value W(t)')
plt.axhline(0, color='black', linestyle='--')
plt.grid(True)
plt.show()

### Part B: Verifying the Distribution of W(T)

The theory says that the position at the end of our simulation, \(W(T)\), should follow a normal distribution \(\mathcal{N}(0, T)\).

Let's test this by running many simulations and plotting a histogram of the final positions.

In [ ]:
N_SIMS = 10000
final_positions = []

for _ in range(N_SIMS):
    t, path = simulate_brownian_motion(TIME_HORIZON, NUM_STEPS)
    final_positions.append(path[-1]) # Get the last value of the path

# Plot the histogram of the final positions
plt.figure(figsize=(12, 6))
plt.hist(final_positions, bins=50, density=True, alpha=0.7, label='Simulated Final Positions')

# Overlay the theoretical Normal PDF
# Mean = 0, Std Dev = sqrt(T)
mu = 0
sigma = np.sqrt(TIME_HORIZON)
x = np.linspace(min(final_positions), max(final_positions), 200)
pdf = norm.pdf(x, mu, sigma)
plt.plot(x, pdf, 'r-', linewidth=2, label=f'Theoretical PDF: N(0, {TIME_HORIZON})')

plt.title(f'Distribution of W(T) at T={TIME_HORIZON} over {N_SIMS} Simulations')
plt.xlabel('Final Position W(T)')
plt.ylabel('Density')
plt.legend()
plt.show()

print(f"Mean of final positions from simulation: {np.mean(final_positions):.4f} (Theoretical: 0)")
print(f"Variance of final positions from simulation: {np.var(final_positions):.4f} (Theoretical: {TIME_HORIZON})")

**Interpretation:**

The simulation results are a stunning confirmation of the theory. The histogram of the final positions of thousands of Brownian paths perfectly matches the theoretical normal distribution curve. The simulated mean and variance are also extremely close to their theoretical values of 0 and T, respectively.

## Summary & Next Steps

In this notebook, we've explored the most important continuous-time stochastic process:
1.  **Brownian Motion** is the continuous-time limit of a random walk.
2.  It is defined by its **continuous paths** and **stationary, independent, normally distributed increments**.
3.  It is a **martingale**.
4.  We can simulate it by taking small discrete steps, where each step is a random draw from a normal distribution scaled by \(\sqrt{\Delta t}\).

Brownian motion is the starting point for stochastic calculus and the modeling of asset prices in finance (e.g., the Black-Scholes model).

In **Weeks 10-11**, we will introduce **Renewal Processes**, which generalize the Poisson process by allowing the time between events to follow any distribution, not just the exponential.